In [1]:
from tkinter import *
from pygame import mixer
import requests
import base64
import json
from tkhtmlview import HTMLLabel
from secrets import *
import sqlite3
con = sqlite3.connect("spotify.db")
c = con.cursor()
import webbrowser
master = Tk()
mixer.init()
mixer.music.load("song.mp3")

class Spotify :
    def __init__(self,master,title,labeltxt):
        self.master = master
        self.master.title = title
        self.master.geometry("600x500")
        self.back = PhotoImage(file = "music.png")
        bgimage = Label(self.master,image = self.back).place(x = 0,y = 0)
        w = Label ( self.master, text = labeltxt,font = ("Arial",30))
        w.grid(row = 0,column = 2)
        self.counter = 0
        y = Button(self.master,text = "Back",command = lambda:Window(master)).place(x=0,y=0)
        
    def search(self,keyword,endpoint,param1,param2):
        global entry
        global olist
        global res
        if endpoint == None :
            c.execute(f'SELECT * FROM {param1} LIMIT {param2}')
            data = c.fetchall()
            print(data)
            f = Frame(self.master)
            f.place(x = 50 , y = 250 , width = 300)
            scroll_bar = Scrollbar(f)

            scroll_bar.pack( side = RIGHT,fill = Y )
            olist = Listbox(f,yscrollcommand = scroll_bar.set)
            olist.pack( fill = BOTH )
            scroll_bar.config(command = olist.yview)
            for i in range(len(data)) :
                olist.insert(i,data[i][2])
        else:
            
            searchtext = entry.get()
            print(searchtext)


            # Step 1 - Authorization 
            url = "https://accounts.spotify.com/api/token"
            headers = {}
            data = {}

            # Encode as Base64

            clientId = "<Enter your client Id>"
            clientSecret =  "<Enter your Client Secret>"

            message = f"{clientId}:{clientSecret}"
            messageBytes = message.encode('ascii')
            base64Bytes = base64.b64encode(messageBytes)
            base64Message = base64Bytes.decode('ascii')

            headers['Authorization'] = f"Basic {base64Message}"
            data['grant_type'] = "client_credentials"

            r = requests.post(url, headers=headers, data=data)

            token = r.json()['access_token']
            #print(token)

            token = r.json()['access_token']

            # Step 2 - Use Access Token to call playlist endpoint


            print(keyword)
            print(searchtext)

            from googlesearch import search
            query = "spotify" + keyword + searchtext 
            for i in search(query,num = 1, stop = 1):
              print(i)
            spotifyId = i.split("/")[-1]
            print(spotifyId)

            #playlistId = "7nYJDIm5nHyNTS2KugaD4w"
            if endpoint == "https://api.spotify.com/v1/artists":
                spotifyUrl = f"{endpoint}/{spotifyId}/top-tracks?market=ES"

            else:
                spotifyUrl = f"{endpoint}/{spotifyId}"

            print(spotifyUrl)

            headers = {
                "Authorization": "Bearer " + token
            }

            res = requests.get(url=spotifyUrl, headers=headers)
            f = Frame(self.master)
            f.place(x = 50 , y = 250 , width = 300)
            scroll_bar = Scrollbar(f)

            scroll_bar.pack( side = RIGHT,fill = Y )
            olist = Listbox(f,yscrollcommand = scroll_bar.set)
            olist.pack( fill = BOTH )
            scroll_bar.config(command = olist.yview)

            print(res.json())
  
            
        
            if endpoint == "https://api.spotify.com/v1/artists":
                
                for i in res.json()["tracks"] :
                    olist.insert(END,i[param1][param2])
                    
                    
                    c.execute('INSERT INTO artisearch(id,Artist_Searched,Songs) VALUES(?,?,?)',(self.counter,searchtext,i[param1][param2]))
                    self.counter+=1
                    con.commit()

            elif endpoint == "https://api.spotify.com/v1/tracks":
                
                for i in (res.json()["artists"]):
                    olist.insert(END,i[param1][param2])
                    
                    webbrowser.open(i[param1][param2])
                    c.execute('INSERT INTO songisearch(id,Song_Searched,Url) VALUES(?,?,?)',(self.counter,searchtext,i[param1][param2]))
                    self.counter+=1
                    con.commit()
            else:
                for i in range(len(res.json()["tracks"]['items'])):

                    olist.insert(END, res.json()["tracks"]['items'][i][param1][param2])
                    
                    c.execute('INSERT INTO playlisearch(id,Playlist_Searched,Songs) VALUES(?,?,?)',(self.counter,searchtext,res.json()["tracks"]['items'][i][param1][param2]))
                    self.counter+=1
                    con.commit()
            
            my_label = HTMLLabel(self.master, html=f"""
            <a style = "text-align : center;" href ="https://open.spotify.com/embed/{keyword}/{spotifyId}" width="50%" height="10" frameborder="0" allowtransparency="true" allow="encrypted-media">LISTEN NOW</a>
            """)
            # Adjust label
            my_label.place(x = 400 , y = 250, width = 100 , height = 27)
               

    def home(self,keyword,endpoint,param1,param2):
        global entry
        label = Label(self.master,text = f"Enter {keyword} : ",font = ("Arial",20)).grid(row = 2,column = 1,pady = 15)
        entry = Entry(self.master)
        entry.grid(row = 2, column = 2, pady = 5)
#         background_img = PhotoImage(file="transparent.png")
        button = Button(self.master,text = "Search",command = lambda:self.search(keyword,endpoint,param1,param2), borderwidth = 0).grid(row = 3,column = 2,pady = 20)
    
    def getData(self,keyword,endpoint,param1,param2):
        button = Button(self.master,text = "Get Top Playlists",command = lambda:self.search(keyword,endpoint,param1,param2), borderwidth = 0).grid(row = 3,column = 1,pady = 20)
        button = Button(self.master,text = "Get Top Artists",command = lambda:self.search(keyword,endpoint,"artisearch",param2), borderwidth = 0).grid(row = 3,column = 2,pady = 20)
        button = Button(self.master,text = "Get Top Songs",command = lambda:self.search(keyword,endpoint,"songisearch",1), borderwidth = 0).grid(row = 3,column = 3,pady = 20)
        
#         c.execute('SELECT * FROM playlisearch LIMIT 10')
#         data = c.fetchall()
#         print(data)
#         f = Frame(self.master)
#         f.place(x = 50 , y = 250 , width = 300)
#         scroll_bar = Scrollbar(f)
  
#         scroll_bar.pack( side = RIGHT,fill = Y )
#         olist = Listbox(f,yscrollcommand = scroll_bar.set)
#         olist.pack( fill = BOTH )
#         scroll_bar.config(command = olist.yview)
#         for i in range(len(data)) :
#             olist.insert(i,data[i][2])

class Window :
    def __init__(self,master):
        self.master = master
        self.master.title = "Spotify"
        self.master.geometry("600x400")
        self.back = PhotoImage(file = "music.png")
        bgimage = Label(self.master,image = self.back).place(x = 0,y = 0)
        
        
        w = Label ( self.master, text = "Choose one of the following :",font = ("Arial",17))

        w.grid(row = 1,column = 2,pady = 10)
        b1 = Button(self.master,text = "Search Playlist",command = self.playlisearch).grid(row = 3,column = 1, pady = 40,padx = 30)
        b2 = Button(self.master,text = "Search Artist", command = self.artisearch).grid(row = 3,column = 2,pady = 40)
        b3 = Button(self.master,text = "Search Song", command = self.songisearch).grid(row = 3,column = 3,pady = 40)
        b4 = Button(self.master,text = "See Top 5 Searched Songs",command = self.user).grid(row = 4,column = 2,pady = 30)
        
#         self.f = Frame(self.master).place(x = 20, y = 60, bordermode = OUTSIDE, height = 50, width = 100)
        b5 = Button(self.master,text = "Play",command = mixer.music.play).grid(row = 7 , column = 0, pady = 40)
        b6 = Button(self.master,text = "Pause",command = mixer.music.pause).grid(row = 7, column = 1, pady = 40)
        b7 = Button(self.master,text = "Stop",command = mixer.music.stop).grid(row = 7, column = 2, pady = 40)
        b8 = Button(self.master,text = "Resume",command = mixer.music.unpause).grid(row = 7 , column = 3, pady = 40)
        
    def playlisearch(self):
        x = Spotify(master,"PlayliSearch","PlayliSearch")
        x.home("playlist","https://api.spotify.com/v1/playlists","track","name")
        
    def artisearch(self):
        x = Spotify(master,"ArtiSearch","ArtiSearch")
        x.home("artist","https://api.spotify.com/v1/artists","album","name")
    
    def songisearch(self):
        x = Spotify(master,"SongiSearch","SongiSearch")
        x.home("track","https://api.spotify.com/v1/tracks","external_urls","spotify")
        
    def user(self):
        x = Spotify(master,"Favourites","Favourites")
        x.getData("Songs",None,"playlisearch",10)
        
#https://api.spotify.com/v1/artists/{id}
#f"https://api.spotify.com/v1/playlists/{playlistId}"
#<iframe src="https://open.spotify.com/embed/album/4PwSDqUwhh4sZUHyi7UzXb" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>
main = Window(master)
master.mainloop()
con.close()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Rock Classics
playlist
Rock Classics
https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U
37i9dQZF1DWXRqgorJj26U
https://api.spotify.com/v1/playlists/37i9dQZF1DWXRqgorJj26U
{'collaborative': False, 'description': 'Rock legends and epic songs that continue to inspire generations.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U'}, 'followers': {'href': None, 'total': 8986897}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWXRqgorJj26U', 'id': '37i9dQZF1DWXRqgorJj26U', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f00000003519fc8771d90f496501a4da3', 'width': None}], 'name': 'Rock Classics', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spo

In [1]:
command = lambda : "Your name"

In [18]:
command()

'Your name'

In [66]:
for i in (res.json()["artists"]):
    print(i["external_urls"]["spotify"])
 
    

https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ


In [2]:
import sqlite3


In [10]:
con = sqlite3.connect("spotify.db")
c = con.cursor()
names = ["Virat","Rohit","Shikhar","KL"]
emails = ["virat@gmail.com","rohit@hotmail.com","shikhar@outlook.com","kl_rahul@outlook.com"]
spotify_ids =[7585639,6284654,5373548,2154664]
for i in range(0,4) :
    c.execute('INSERT INTO user(id,Name,EmailID,SpotifyID) VALUES (?,?,?,?)',(i+2,na))
con.commit()
con.close()

IntegrityError: UNIQUE constraint failed: user.id

In [2]:
import sqlite3
con = sqlite3.connect("spotify.db")
c = con.cursor()
c.execute('CREATE TABLE playlisearch(id,Playlist_Searched,Songs)')
con.commit()
con.close()

In [3]:
import sqlite3
con = sqlite3.connect("spotify.db")
c = con.cursor()
c.execute('CREATE TABLE artisearch(id,Artist_Searched,Songs)')
con.commit()
con.close()

In [4]:
import sqlite3
con = sqlite3.connect("spotify.db")
c = con.cursor()
c.execute('CREATE TABLE songisearch(id,Song_Searched,Url)')
con.commit()
con.close()

In [3]:
import subprocess

In [ ]:
subprocess.call(["afplay","song.mp3"])

In [4]:
pip install pygame

     |████████████████████████████████| 6.9 MB 6.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tkhtmlview

  Created wheel for tkhtmlview: filename=tkhtmlview-0.1.0.post1-py3-none-any.whl size=9418 sha256=eb33c0426ad9ad01720dca1502769444120b48a2b31d6e548ffb98e3100c58c4
  Stored in directory: /Users/nitindhingra/Library/Caches/pip/wheels/8d/99/3d/f0239b92cc6e68c5cd5e686957721d5cd4ecbfe0bba4a96445
Successfully built tkhtmlview
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tkinter import *
from tkhtmlview import HTMLLabel
  
# Create Object
root = Tk()
  
# Set Geomerty
root.geometry("400x400")
  
# Add label
my_label = HTMLLabel(root, html="""
        <h1>MUSIC  PLAYER</h1>
        <a href ="https://open.spotify.com/embed/playlist/0LjVMQm4TftvPoWD6Tzows" width="100%" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media">PLAY</a>
    """)
  
# Adjust label
my_label.pack(pady=20, padx=20)
  
# Execute Tkinter
root.mainloop()